<a href="https://colab.research.google.com/github/YouJoohyung/myfirst-project/blob/0607_%EC%9E%A5%EB%B3%91%ED%83%81pf/Lab2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision.datasets import MNIST
from torchvision.transforms import Compose,ToTensor

from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import PIL

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Connect to local google drive & settings for export/import models

In [13]:
drive.mount('/content/gdrive')
!mkdir ./gdrive/'My Drive'/MNIST_models/

Mounted at /content/gdrive
mkdir: cannot create directory ‘./gdrive/My Drive/MNIST_models/’: File exists


#### DNN model

In [14]:
class MNISTDNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTDNN,self).__init__()
        self.fc1 = nn.Linear(IMG_SIZE*IMG_SIZE,32)
        self.BN1 = torch.nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32,10)
    
    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.fc2(x)
        x = torch.softmax(x,dim=-1)
        return x    

#### CNN model

In [15]:
class MNISTCNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTCNN,self).__init__()
        self.conv1 = nn.Conv2d(1,8,5,stride=2)
        self.BN1 = torch.nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8,8,5,stride=2)
        self.BN2 = torch.nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8,8,3,stride=1)
        self.fc = nn.Linear(8*2*2,10)
    
    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.BN2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = x.view(-1,8*2*2)
        x = self.fc(x)
        x = torch.softmax(x,dim=-1)
        return x

#### Util function for calculating accuracy

In [16]:
def compute_acc(argmax,y):
    count = 0
    for i in range(len(argmax)):
        if argmax[i]==y[i]:
            count+=1
    return count / len(argmax)

#### hyperparameters & datasets

In [17]:
IMG_SIZE = 28
BATCH_SIZE = 256
LEARNING_RATE = 0.001
NUM_EPOCHES = 30

In [18]:
transforms = Compose([
    ToTensor(),
])

trainset = MNIST('/content/gdrive/MyDrive/MNIST_models/',train=True,transform=transforms,download=True)
testset = MNIST('/content/gdrive/MyDrive/MNIST_models/',train=False,transform=transforms,download=True)

args = {
    'num_workers' : 1,
    'batch_size' : BATCH_SIZE,
    'shuffle' : True,
}

train_loader = DataLoader(trainset,**args)
test_loader = DataLoader(testset,**args)

100%|██████████| 9912422/9912422 [00:00<00:00, 176320680.78it/s]

Extracting /content/gdrive/MyDrive/MNIST_models/MNIST/raw/train-images-idx3-ubyte.gz to /content/gdrive/MyDrive/MNIST_models/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 154312985.76it/s]

Extracting /content/gdrive/MyDrive/MNIST_models/MNIST/raw/train-labels-idx1-ubyte.gz to /content/gdrive/MyDrive/MNIST_models/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 88070235.67it/s]


Extracting /content/gdrive/MyDrive/MNIST_models/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/gdrive/MyDrive/MNIST_models/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 7687864.72it/s]

Extracting /content/gdrive/MyDrive/MNIST_models/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/gdrive/MyDrive/MNIST_models/MNIST/raw



####Training part(DNN)

In [19]:
model = MNISTDNN(IMG_SIZE).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0
    
    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        loss = loss_fn(y_, y.cuda())
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()
        
    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss))
    if epoch % 2 == 1:
        x,y = next(iter(test_loader))
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())
        
        print("Acc(val) : {}".format(test_acc))

torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/DNN.pt")

number of parameters : 25514
Epoch 1, Loss(train) : 407.6065183877945
Epoch 2, Loss(train) : 368.231654047966
Acc(val) : 0.9296875
Epoch 3, Loss(train) : 361.77316319942474
Epoch 4, Loss(train) : 358.40818321704865
Acc(val) : 0.9609375
Epoch 5, Loss(train) : 356.22821593284607
Epoch 6, Loss(train) : 354.5317679643631
Acc(val) : 0.96484375
Epoch 7, Loss(train) : 353.2864907979965
Epoch 8, Loss(train) : 352.3718651533127
Acc(val) : 0.96875
Epoch 9, Loss(train) : 351.6593829393387
Epoch 10, Loss(train) : 350.9839265346527
Acc(val) : 0.95703125
Epoch 11, Loss(train) : 350.51428508758545
Epoch 12, Loss(train) : 349.9280000925064
Acc(val) : 0.9609375
Epoch 13, Loss(train) : 349.71957993507385
Epoch 14, Loss(train) : 349.4683338403702
Acc(val) : 0.94921875
Epoch 15, Loss(train) : 349.0863035917282
Epoch 16, Loss(train) : 348.7412737607956
Acc(val) : 0.9765625
Epoch 17, Loss(train) : 348.631089925766
Epoch 18, Loss(train) : 348.3083356618881
Acc(val) : 0.96875
Epoch 19, Loss(train) : 348.16791

In [20]:
model_test = MNISTDNN(IMG_SIZE).cuda()
model_test.load_state_dict(torch.load("/content/gdrive/My Drive/MNIST_models/DNN.pt"))
model_test.eval()
x,y = next(iter(test_loader))
x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
y_ = model_test(x)
_, argmax = torch.max(y_,dim=-1)
test_acc = compute_acc(argmax,y.numpy())

print("Acc(test) : {}".format(test_acc))

Acc(test) : 0.98046875


#### Training part(CNN)

In [ ]:
model = MNISTCNN(IMG_SIZE).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0
    
    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda()
        y_ = model(x)
        loss = loss_fn(y_, y.cuda())
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()
        
    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss))
    if epoch % 2 == 1:
        model.eval()
        
        x,y = next(iter(test_loader))
        x = x.cuda()
        y_ = model(x)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())
        
        print("Acc(test) : {}".format(test_acc))
        
        model.train()

torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/CNN.pt")

number of parameters : 2762
Epoch 1, Loss(train) : 431.99150371551514
Epoch 2, Loss(train) : 365.27305698394775
Acc(test) : 0.94140625
Epoch 3, Loss(train) : 358.12150156497955
Epoch 4, Loss(train) : 355.04672944545746
Acc(test) : 0.97265625
Epoch 5, Loss(train) : 353.2219293117523
Epoch 6, Loss(train) : 352.014816403389
Acc(test) : 0.9609375
Epoch 7, Loss(train) : 351.14613676071167
Epoch 8, Loss(train) : 350.4977465867996
Acc(test) : 0.97265625
Epoch 9, Loss(train) : 350.0155040025711
Epoch 10, Loss(train) : 349.5600552558899
Acc(test) : 0.9921875
Epoch 11, Loss(train) : 349.28144812583923
Epoch 12, Loss(train) : 348.8999457359314
Acc(test) : 0.984375
Epoch 13, Loss(train) : 348.7143303155899
Epoch 14, Loss(train) : 348.36587512493134
Acc(test) : 0.98828125
Epoch 15, Loss(train) : 348.1851484775543
Epoch 16, Loss(train) : 348.1170176267624
Acc(test) : 0.9765625
Epoch 17, Loss(train) : 347.8389639854431
Epoch 18, Loss(train) : 347.79782915115356
Acc(test) : 0.9765625
Epoch 19, Loss(tr

In [ ]:
model_test = MNISTCNN(IMG_SIZE).cuda()
model_test.load_state_dict(torch.load("/content/gdrive/My Drive/MNIST_models/CNN.pt"))
model_test.eval()
x,y = next(iter(test_loader))
x = x.cuda()
y_ = model_test(x)
_, argmax = torch.max(y_,dim=-1)
test_acc = compute_acc(argmax,y.numpy())

print("Acc(test) : {}".format(test_acc))